**Direct Path I/O 특징**

**Direct PATH I/O는 버퍼 캐시를 경유하지 않고 곧바로 데이터 블록을 읽고 쓸 수 있는 기능.**

- 일반적으로 I/O 성능을 향상시키기 위해 버퍼 캐시를 경유.

- 대용량 데이터를 읽고 쓰거나 재사용 가능성이 없는 임시 세그먼트 블록들을 일고 쓸 때는 버퍼 캐시를 경유하지 않는 것이 유리.

- 오라클은 버퍼 캐시를 경유하지 않고 곧바로 데이터 블록을 읽고 쓸 수 있는 Direct PATH I/O 기능을 제공.

# **Direct Path I/O가 작동하는 경우**

- Temp 세그먼트 블록들을 읽고 쓸 때
- 병렬 쿼리로 Full Scan을 수행 할 때
- Nocache 옵션을 지정한 LOB 컬럼을 읽을 때
- Direct 옵션을 지정하고 export를 수행할 때
- Parallel DML을 수행할 때
- Direct Path Insert를 수행할 때


# **Direct Path Read/Write Temp**

- 데이터를 정렬할 때는, PGA 메모리에 할당되는 Sort 를 이용하고 정렬할 데이터가 많아 Sort Area가 부족해지면 Temp 테이블스페이스를 이용.
  - 이때 Sort Area에 정렬된 데이터를 Temp 테이블 스페이스에 쓰고 이를 다시 읽을 때 Direct Path I/O방식을 사용=.
- direct path write temp 와 direct path read temp 이벤트
  - 이 과정에서 I/O Call이 완료될 때까지 대기가 발생.

# **Direct Path Read**

- 병렬 쿼리로 Full Scan을 수행할 때도 Direct Path Read 방식을 사용.
- direct path read 이벤트 : Direct Path Read 과정에서 읽기 Call이 완료될 때까지 대기가 발생.
- 버퍼 캐시에만 기록된 변경사항이 아직 데이터파일에 기록되지 않는 상태에서 데이터 파일을 직접 읽으면 정합성에 문제가 생긴다.
  - 그러므로 Direct Path Read를 수행하려면 메모리와 디스크간 동기화를 먼저 수행함으로써 Dirty 버퍼를 해소해야함.
  - 10g R2이전에는 CKPT를 발생하여 버퍼 캐시 전체를 데이터파일에 기록 했지만 10g R2부터는 병렬 쿼리와 관련된 세그먼트만 동기화를 수행.

# **Direct Path Write**

- 병렬로 DML을 수행하거나 Direct Path Insert 방식으로 데이터를 insert 할 때 사용.
  - 이 과정에서 I/O Call이 발생할 때마다 direct path write 대기 이벤트가 발생.

**Conventional insert 시 Freelist를 통해 데이터를 삽입할 블록을 할당받는 과정**

- Freelist를 조회하면서 Random 액세스 방식으로 버퍼 캐시에서 해당 블록을 찾음.
- 없으면 데이터파일에서 읽어 캐시에 적재한 후에 데이터를 삽입한다. 이 때 대량의 데이터를 insert 할 때 매우 느림.

### **1. Direct Path Insert**

- Freelist를 참조하지 않고 테이블 세그먼트 또는 각 파티션 세그먼트의 HWM 바깥 영역에 데이터를 순차적으로 입력.
- Redo와 Undo 엔트리를 로깅하지 않도록 옵션을 줄 수도 있어 훨씬 빠름.

- Direct Path Insert 방식으로 데이터를 입력하거나 병렬 방식으로 DML을 수행하면 Exclusive 모드 테이블 Lock이 걸린다. 즉, 다른 트랜잭션이 수행할 수 없음.
 - 트랜잭션이 빈번한 때에는 사용하지 말것.

[ex] Exclusive 모드 TM LOCK이 걸림


```sql
alter session enable parallel dml;
Delete /*+ parallel(b 4) */ from big_table b;
```

### **2. Direct Path Insert 방식으로 데이터를 입력하는 방식**
1. Insert...select 문장에 /+ append / 힌트 사용
2. 병렬모드로 insert
3. Direct 옵션을 지정하고 SQL*Loader(sqlldr)로 데이터를 로드
4. CTAS(create table... as select) 문장을 수행